In [6]:
import sys
def stdwrite(buff):
    sys.stdout.write('\r' + buff)
    sys.stdout.flush()

In [ ]:

from importlib import reload
from simulation.simulation_builder import graph_builder as gb
from simulation.architectures.mnist_architectures import cnn_mnist_architecture
from tensorflow.python.client import device_lib
import sys
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
reload(gb)
g = gb.GraphBuilder(cnn_mnist_architecture,
                   learning_rate=0.01,
                   noise_list=[0.9, 0.8, 0.7, 0.6],
                   noise_type='dropout',
                   name='test')
#device_lib.list_local_devices()

def stdwrite(buff):
    sys.stdout.write('\r' + buff)
    sys.stdout.flush()

In [ ]:
batch_size = 50
with g._graph.as_default():
    from tensorflow.examples.tutorials.mnist import input_data
    data_path = 'simulation/data/mnist/'
    mnist = input_data.read_data_sets(data_path)
    test_feed_dict = g.create_feed_dict(mnist.test.images,
                                       mnist.test.labels,
                                       test=True)
    step = 0
    n_iters = n_iters = mnist.train.num_examples // batch_size
    with tf.Session() as sess:
        sess.run(g.variable_initializer)
        for epoch in range(10):
            for it in range(n_iters):
                step += 1
                X_batch, y_batch = mnist.train.next_batch(batch_size)

                evaluated = sess.run(g.get_train_ops(),
                                    feed_dict=g.create_feed_dict(X_batch,
                                                                y_batch))
                if it % 200 == 0:
                    evaluated = sess.run(g.get_train_ops(test=True),
                                        feed_dict=test_feed_dict)
                    loss = g.extract_evaluated_tensors(evaluated, 'loss')
                    buff = ''
                    for l in loss:
                        buff = buff + ' ,' + str(l)
                    stdwrite(buff)
                    #g.update_noise_vals(evaluated)
    

In [2]:
import tensorflow as tf
from simulation.simulation_builder import graph_builder as gb
from importlib import reload
from simulation.architectures.mnist_architectures import nn_mnist_architecture
reload(gb)
g = gb.GraphBuilder(nn_mnist_architecture,
                   learning_rate=0.01,
                   noise_list=[0.001, 0.003, 0.004, 0.009],
                   noise_type='random_normal',
                   name='test')

1


In [7]:
batch_size = 50
with g._graph.as_default():
    from tensorflow.examples.tutorials.mnist import input_data
    data_path = 'simulation/data/mnist/'
    mnist = input_data.read_data_sets(data_path)
    test_feed_dict = g.create_feed_dict(mnist.test.images,
                                       mnist.test.labels,
                                       test=True)
    step = 0
    n_iters = n_iters = mnist.train.num_examples // batch_size
    with tf.Session() as sess:
        sess.run(g.variable_initializer)
        for epoch in range(10):
            for it in range(n_iters):
                step += 1
                X_batch, y_batch = mnist.train.next_batch(batch_size)

                evaluated = sess.run(g.get_train_ops(),
                                    feed_dict=g.create_feed_dict(X_batch,
                                                                y_batch))
                if it % 200 == 0:
                    evaluated = sess.run(g.get_train_ops(test=True),
                                        feed_dict=test_feed_dict)
                    loss = g.extract_evaluated_tensors(evaluated, 'loss')
                    buff = ''
                    for l in loss:
                        buff = buff + ' ,' + str(l)
                    stdwrite(buff)
                    g.update_noise_vals(evaluated)
    

Extracting simulation/data/mnist/train-images-idx3-ubyte.gz
Extracting simulation/data/mnist/train-labels-idx1-ubyte.gz
Extracting simulation/data/mnist/t10k-images-idx3-ubyte.gz
Extracting simulation/data/mnist/t10k-labels-idx1-ubyte.gz
 ,2.5708725 ,2.6581137 ,2.6045806 ,2.6286843

SystemExit: 

c:\users\vladp\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
raise ValueError("""This is test for me""")

ValueError: This is test for me